# README
This notebook is used to split the dataset.

In [1]:
import numpy as np
import pickle
import pandas as pd
import numpy as np
import random
import json
import os
from batteryml.data.battery_data import BatteryData
import matplotlib.pyplot as plt
from tqdm import tqdm
random.seed(2024)
np.random.seed(2024)

# ISU_ILCC dataset

In [2]:
# Here is the train_ids provided by BatteryML
data_path = '/data/trf/python_works/Battery-LLM/dataset/ISU_ILCC'
tmp_files = os.listdir(data_path)
life_labels = json.load(open('/data/trf/python_works/Battery-LLM/dataset/ISU-ILCC_labels.json'))
files = [i for i in tmp_files if i in life_labels]

test_ratio = 0.2
testing_set = random.sample(files, int(len(files)*0.2))
training_set = [i for i in files if i not in testing_set]

val_ratio = 0.2

validation_set= random.sample(training_set, int(len(training_set)*0.25))
training_set = [i for i in training_set if i not in validation_set]


assert len(training_set) + len(validation_set) + len(testing_set) == len(files)

print(f'training_set: {len(training_set)} {training_set}')
print(f'Val set: {len(validation_set)} {validation_set}')
print(f'Test set: {len(testing_set)} {testing_set}')

training_set: 144 ['ISU-ILCC_G23C3.pkl', 'ISU-ILCC_G64C4.pkl', 'ISU-ILCC_G50C1.pkl', 'ISU-ILCC_G36C2.pkl', 'ISU-ILCC_G7C4.pkl', 'ISU-ILCC_G16C2.pkl', 'ISU-ILCC_G13C2.pkl', 'ISU-ILCC_G29C3.pkl', 'ISU-ILCC_G41C3.pkl', 'ISU-ILCC_G39C4.pkl', 'ISU-ILCC_G36C4.pkl', 'ISU-ILCC_G28C3.pkl', 'ISU-ILCC_G18C1.pkl', 'ISU-ILCC_G58C1.pkl', 'ISU-ILCC_G8C2.pkl', 'ISU-ILCC_G16C3.pkl', 'ISU-ILCC_G27C2.pkl', 'ISU-ILCC_G22C1.pkl', 'ISU-ILCC_G43C1.pkl', 'ISU-ILCC_G53C2.pkl', 'ISU-ILCC_G32C4.pkl', 'ISU-ILCC_G44C3.pkl', 'ISU-ILCC_G38C1.pkl', 'ISU-ILCC_G61C3.pkl', 'ISU-ILCC_G43C2.pkl', 'ISU-ILCC_G20C2.pkl', 'ISU-ILCC_G17C1.pkl', 'ISU-ILCC_G33C4.pkl', 'ISU-ILCC_G60C1.pkl', 'ISU-ILCC_G27C3.pkl', 'ISU-ILCC_G45C3.pkl', 'ISU-ILCC_G18C4.pkl', 'ISU-ILCC_G50C4.pkl', 'ISU-ILCC_G1C1.pkl', 'ISU-ILCC_G19C4.pkl', 'ISU-ILCC_G49C4.pkl', 'ISU-ILCC_G3C3.pkl', 'ISU-ILCC_G19C2.pkl', 'ISU-ILCC_G52C4.pkl', 'ISU-ILCC_G31C1.pkl', 'ISU-ILCC_G47C2.pkl', 'ISU-ILCC_G43C4.pkl', 'ISU-ILCC_G47C1.pkl', 'ISU-ILCC_G40C2.pkl', 'ISU-ILCC_G20C4.p

# Data check

In [3]:
cell_name = 'ISU-ILCC_G42C4.pkl'
data = pickle.load(open(f'../dataset/ISU_ILCC/{cell_name}', 'rb'))
cycle_data = data['cycle_data']
nominal_capacity = data['nominal_capacity_in_Ah']
need_keys = ['current_in_A', 'voltage_in_V', 'charge_capacity_in_Ah', 'discharge_capacity_in_Ah', 'time_in_s']
SOC_interval = data['SOC_interval'] # get the charge and discharge soc interval
SOC_interval = SOC_interval[1] - SOC_interval[0]
fig = plt.figure(figsize=(6,4))
total_xs = []
total_SOHs = []
for correct_cycle_index, sub_cycle_data in enumerate(cycle_data):
    soh = max(sub_cycle_data['discharge_capacity_in_Ah']) / nominal_capacity / SOC_interval
    total_SOHs.append(soh)
    total_xs.append(correct_cycle_index+1)
    cycle_df = pd.DataFrame()
    for key in need_keys:
        cycle_df[key] = sub_cycle_data[key]    
    cycle_df['cycle_number'] = correct_cycle_index + 1
    cycle_df['cycle_number'] = correct_cycle_index + 1
    cycle_df.loc[cycle_df['charge_capacity_in_Ah']<0] = np.nan
    cycle_df.bfill(inplace=True) # deal with NaN

    
    
    cycle_df = cycle_df.loc[cycle_df['current_in_A']<0]
    voltage_records = cycle_df['voltage_in_V'].values
    current_records = cycle_df['current_in_A'].values
    charge_capacity_records = cycle_df['charge_capacity_in_Ah'].values
    discharge_capacity_records = cycle_df['discharge_capacity_in_Ah'].values
    plt.plot(discharge_capacity_records, voltage_records, marker='o')

plt.show()

fig = plt.figure(figsize=(6,4))
plt.scatter(total_xs, total_SOHs)
plt.title(cell_name)
plt.xlabel('Cycle number')
plt.ylabel('SOH')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/ISU_ILCC/ISU-ILCC_G42C4.pkl'